# Filtering - Pre-Processing of the SEEG Signal (1/2)
This notebook presents the **pre-processing stage 1** the SEEG signal goes through before being fed to the SNN. The pre-processing stages are as follows:
1. **Filtering**: The SEEG signal is bandpass filtered to remove noise and artifacts. The bandpass filter is designed using the Butterworth filter and, since we are working with *iEEG*, the signal is filtered in the ripples and FR bands. The co-occurrence of HFOs in both bands is an optimal prediction of post-surgical seizure freedom by defining an optimal "HFO area" or EZ zone.
2. **Signal-to-Spike Conversion**: To interface and communicate with the silicon neurons in the SNN, the SEEG signal must be converted to spikes.

## Filtering
Depending on the EEG modality, the signal is filtered in different frequency bands. In this case, since we are handling *iEEG* or *sEEG* data, the signal is filtered in both the ripples (80-250Hz) and FR bands (250-500Hz). The co-occurrence of HFO in these bands represents an optimal prediction of post-surgical seizure freedom by defining an optimal "HFO area" or EZ zone.

The filter is implemented in different ways depending on the setup it will run on.
1. **Neuromorphic Hardware**: The filter is implemented using analog filters. 
2. **Software Simulation**: *Butterworth filters* are utilized since they are a good approximation of the tuned *Tow-Thomas* architectures implemented in hardware.

The frequency response of the *Butterworth filter* is maximally flat in the passband and rolls of towards 0 in the stopband.

### Check WD (change if necessary) and file loading

In [74]:
# Show current directory
import os
curr_dir = os.getcwd()
print(curr_dir)

# Check if the current WD is the file location
if "/src/hfo/filter" not in os.getcwd():
    # Set working directory to this file location
    file_location = f"{os.getcwd()}/thesis-lava/src/hfo/filter"
    print("File Location: ", file_location)

    # Change the current working Directory
    os.chdir(file_location)

    # New Working Directory
    print("New Working Directory: ", os.getcwd())

PATH_TO_FILE = '' # 'src/hfo/'  # This is needed if the WD is not the same as the file location

/home/monkin/Desktop/feup/thesis/thesis-lava/src/hfo/filter


In [75]:
import numpy as np
import math

INPUT_FILE_COMMON = "seeg_csl"  # "seeg_synthetic_humans"
seeg_file_name = f"{INPUT_FILE_COMMON}.npy"   # "seeg_synthetic_humans.npy"
markers_seeg_file_name = f"{INPUT_FILE_COMMON}_markers.npy"

IS_CLINICAL = True
OUTPUT_PATH = "/clinical" if IS_CLINICAL else "/synthetic"
OUT_FILE_PREFIX = "filtered_seeg_csl_ch" if IS_CLINICAL else "filtered_seeg_ch"

recorded_data = np.load(f"{PATH_TO_FILE}data/{seeg_file_name}")

print("Data shape: ", recorded_data.shape)
print("First time steps: ", recorded_data[:10])

Data shape:  (129239, 86)
First time steps:  [[  1.0633698  34.293705   -5.3168535 ... -54.76359    13.557976
  -24.457527 ]
 [  1.8608971  35.888763   -6.380224  ... -57.687862   15.684719
  -25.78674  ]
 [ -1.8608971  37.21797    -7.1777525 ... -62.738873   19.406517
  -22.862473 ]
 ...
 [ -1.3292122  34.559547   -8.772809  ... -72.57505    20.204044
   -2.9242706]
 [ -0.7975273  37.74966   -13.026291  ... -67.78989    16.216404
   -1.8608971]
 [ -1.3292122  34.027863   -9.304494  ... -68.85326    14.355507
    0.5316849]]


## Define the Filter

In [76]:
from scipy.signal import butter, lfilter

# ================================================================ #
# ============ Butterworth Filter Coefficients =================== #
# ================================================================ #
def butter_bandpass(lowcut, highcut, sampling_freq, order=5):
    """
    This function is used to generate the coefficients for lowpass, highpass and bandpass
    filtering for Butterworth filters.
    @lowcut, highcut (int): cutoff frequencies for the bandpass filter
    @sampling_freq (float): sampling_frequency frequency of the wideband signal
    @order (int): filter order

    - return b, a (float): filtering coefficients that will be applied on the wideband signal
    """
    nyq = 0.5 * sampling_freq   # Nyquist frequency
    low = lowcut / nyq          # Normalizing the cutoff frequencies
    high = highcut / nyq        # Normalizing the cutoff frequencies

    return butter(order, [low, high], btype='band')    

# ================================================================ #
# ====================== Butterworth Filters ===================== #
# ================================================================ #
def butter_bandpass_filter(data, lowcut, highcut, sampling_freq, order=5):
    """
    This function applies the filtering coefficients calculated above to the wideband signal (original signal).
    @data (array): Array with the amplitude values of the wideband signal.
    @lowcut, highcut (int): cutoff frequencies for the bandpass filter.
    @sampling_freq (float): sampling frequency of the original signal.
    @order (int): filter order.

    - return (array): Array with the amplitude values of the filtered signal.
    """
    coef_b, coef_a = butter_bandpass(lowcut, highcut, sampling_freq, order)

    return lfilter(coef_b, coef_a, data)
    

## Define Global Parameters of the Experiment

In [77]:
from utils.input import SAMPLING_RATE, X_STEP

sampling_rate = SAMPLING_RATE    # 2048 Hz
x_step = X_STEP  # 0.48828125 ms

num_samples = recorded_data.shape[0]    # 2048 * 120 = 245760
num_channels = recorded_data.shape[1]   # 960
input_duration = (num_samples / sampling_rate) * 1000

print(f"Input Duration: {input_duration} ms")

Input Duration: 63104.98046875 ms


### Extract a window of channels from the SEEG data
Let's define the window first.

If we want to extract a single channel, set the variable `is_single_channel` to `True` and the variable `min_channel_idx` to the desired channel number.

In [78]:
is_single_channel = False   # Set to True if you want to use only one channel

# Define the window of channels to be used
BRAIN_REGION_IDX = 0
BRAIN_REGION_OFFSET = BRAIN_REGION_IDX * 120
SNR_OFFSET = 90     # Choose the highest SNR (channels 90-120)
min_channel_idx = 30  # BRAIN_REGION_OFFSET + SNR_OFFSET
max_channel_idx = min_channel_idx + 30

if is_single_channel:
    # Set the window to size 1
    max_channel_idx = min_channel_idx + 1

In [79]:
from utils.io import preview_np_array
seeg_window = recorded_data[:, min_channel_idx:max_channel_idx]

preview_np_array(seeg_window, "SEEG Window")

SEEG Window Shape: (129239, 30).
Preview: [[ -55.56112    -59.814606   -10.102022   -18.077301    12.228763  ...
   -32.16696     74.7018    -114.04651    -18.874832    29.774384 ]
 [ -56.890335   -60.61213    -10.102022   -20.469887    10.102021  ...
   -34.027863    69.38493   -108.9955     -20.469894    30.837753 ]
 [ -58.219543   -60.612133    -9.304494   -17.81146     14.355505  ...
   -32.964493    69.119095  -110.32471    -23.394157    35.09124  ]
 [ -57.42202    -60.34629    -11.697078   -20.73573     14.88719   ...
   -36.42045     69.38493   -110.59056    -21.001572    35.091232 ]
 [ -56.890335   -59.28292    -14.88719    -21.001572    14.355504  ...
   -38.281345    64.068085  -109.79303    -21.799103    36.420456 ]
 ...
 [ -49.44674    -34.293705    11.431236    20.20404     17.279776  ...
    39.078873    34.825386   -78.68943    -18.343143     7.1777525]
 [ -46.52247    -34.825394    11.165398    19.67236     15.418875  ...
    37.217976    37.749657   -79.221115   -14.88

## Apply the Butterworth filter to each channel

In [80]:
# Apply the Butterworth filter to the window of channels in the Ripple Band
ripple_lowcut_freq = 80
ripple_highcut_freq = 250
BUTTER_FILTER_ORDER = 9

ripple_band_seeg_window = [ butter_bandpass_filter(seeg_window[:, i], ripple_lowcut_freq, ripple_highcut_freq, sampling_rate, BUTTER_FILTER_ORDER) for i in range(seeg_window.shape[1]) ]
ripple_band_seeg_window = np.array(ripple_band_seeg_window).T
preview_np_array(ripple_band_seeg_window, "Ripple Band SEEG Window", edge_items=3)

Ripple Band SEEG Window Shape: (129239, 30).
Preview: [[-8.30193599e-05 -8.93749147e-05 -1.50944298e-05 ... -1.70408162e-04
  -2.82027524e-05  4.44888495e-05]
 [-1.21216938e-03 -1.30402062e-03 -2.20033332e-04 ... -2.47651320e-03
  -4.13498278e-04  6.50108234e-04]
 [-8.47334814e-03 -9.10609068e-03 -1.53379209e-03 ... -1.72211275e-02
  -2.90710783e-03  4.55367995e-03]
 ...
 [-3.83818553e-01  3.74478269e-01  1.82346539e-01 ...  5.86743965e-01
   2.52448301e+00 -1.09098293e+00]
 [-5.64172868e-01  5.83872137e-01 -6.55029530e-02 ... -5.57722657e-01
   3.59086578e+00 -2.20422319e+00]
 [-5.98851221e-01  7.19539413e-01 -2.96924496e-01 ... -1.52184342e+00
   3.87510043e+00 -2.63788951e+00]]


In [81]:
# Apply the Butterworth filter to the window of channels in the Fast Ripple Band
fr_lowcut_freq = 250
fr_highcut_freq = 500

fr_band_seeg_window = [ butter_bandpass_filter(seeg_window[:, i], fr_lowcut_freq, fr_highcut_freq, sampling_rate, BUTTER_FILTER_ORDER) for i in range(seeg_window.shape[1]) ]
fr_band_seeg_window = np.array(fr_band_seeg_window).T
preview_np_array(fr_band_seeg_window, "FR Band SEEG Window", edge_items=3)

FR Band SEEG Window Shape: (129239, 30).
Preview: [[-1.60660266e-03 -1.72959628e-03 -2.92109590e-04 ... -3.29776340e-03
  -5.45783744e-04  8.60954651e-04]
 [-1.12527184e-02 -1.20958544e-02 -2.03896062e-03 ... -2.28727365e-02
  -3.85575994e-03  6.04031697e-03]
 [-2.14036610e-02 -2.28729889e-02 -3.81274635e-03 ... -4.23232028e-02
  -7.57340445e-03  1.16431601e-02]
 ...
 [ 1.17118922e+00 -3.52740623e-01 -9.42723819e-01 ...  1.04041108e-01
  -4.12746111e-01  1.47455335e-02]
 [ 3.77408302e-01  5.62804334e-01 -1.36302207e+00 ...  7.55420292e-01
  -3.70504660e-01 -1.01931771e+00]
 [-8.05914205e-01  8.92573648e-01 -4.09878669e-01 ...  8.95009709e-01
  -6.29180359e-01 -7.11263606e-01]]


Apply the Butterworth filter in the combined Ripple+FR Band

In [82]:
# Apply the Butterworth filter to the window of channels in the Combined Ripple and Fast Ripple Band
both_band_seeg_window = [ butter_bandpass_filter(seeg_window[:, i], ripple_lowcut_freq, fr_highcut_freq, sampling_rate, BUTTER_FILTER_ORDER) for i in range(seeg_window.shape[1]) ]
both_band_seeg_window = np.array(both_band_seeg_window).T
preview_np_array(both_band_seeg_window, "Both Bands SEEG Window", edge_items=3)

Both Bands SEEG Window Shape: (129239, 30).
Preview: [[-0.07076982 -0.07618761 -0.01286724 ... -0.14526437 -0.02404143
   0.0379245 ]
 [-0.66374743 -0.71375383 -0.12037353 ... -1.3525201  -0.22694013
   0.35613961]
 [-2.62400238 -2.81551241 -0.47288795 ... -5.29163038 -0.908183
   1.41485784]
 ...
 [ 0.26414082 -0.07446556  1.14011617 ... -2.08017214  2.84187845
   0.02161914]
 [ 1.11657756 -0.25260712  0.49740641 ... -3.41669097  2.57734347
  -0.13053429]
 [ 0.5242394  -0.25160076 -1.06733936 ... -3.91685808  0.81962739
  -0.06098256]]


## Import the Markers (Annotated Events) 
The markers are stored in a numpy array of shape (num_channels, events):
- Each row represents the events of a channel
- Each event is composed of the following 3 fields (Label, Position, Shape)

In [83]:
markers = np.load(f"{PATH_TO_FILE}data/{markers_seeg_file_name}", allow_pickle=True)

preview_np_array(markers, "Markers", edge_items=3)

Markers Shape: (86,).
Preview: [list([array([],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])])
 list([array([],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])])
 list([array([],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])])
 ...
 list([array([],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])])
 list([array([],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])])
 list([array([],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])])]


### Define the set of channels the markers will be extracted from

In [84]:
channels_used = set(range(min_channel_idx, max_channel_idx))
print("Channels used: ", channels_used)

Channels used:  {30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59}


## Visualize the filtered signals

In [85]:
# Interactive Plot for the HFO detection
# bokeh docs: https://docs.bokeh.org/en/2.4.1/docs/first_steps/first_steps_1.html

from utils.line_plot import create_fig  # Import the function to create the figure
from bokeh.models import Range1d

# Define the x and y values
# Should the first input start at 0 or x_step?
# TODO: is it okay to create a range with floats?
x = [val for val in np.arange(x_step, input_duration + x_step, x_step)] 

## Create the Plot

In [86]:
# Create the plot
# List of tuples containing the y values and the legend label
hfo_y_arrays = []

PLOT_RIPPLE_BAND = False
PLOT_FR_BAND = False
PLOT_BOTH_BAND = True

if is_single_channel:
    # Add the Ripple and FR bands of the single channel
    hfo_y_arrays.append((ripple_band_seeg_window[:, 0], f"Ripple Band Ch. {min_channel_idx}"))
    hfo_y_arrays.append((fr_band_seeg_window[:, 0], f"Fast Ripple Band Ch. {min_channel_idx}"))
else:
    # Add the Ripple, FR and both bands of each channel in the range defined below
    min_hfo_idx = 0
    max_hfo_idx = 8
    if PLOT_RIPPLE_BAND:
        for hfo_idx in range(min_hfo_idx, max_hfo_idx, 1):
            hfo_y_arrays.append((ripple_band_seeg_window[:, hfo_idx], f"Ripple Band Ch. {min_channel_idx + hfo_idx}"))
    if PLOT_FR_BAND:
        for hfo_idx in range(min_hfo_idx, max_hfo_idx, 1):
                hfo_y_arrays.append((fr_band_seeg_window[:, hfo_idx], f"Fast Ripple Band Ch. {min_channel_idx + hfo_idx}"))
    if PLOT_BOTH_BAND:
        for hfo_idx in range(min_hfo_idx, max_hfo_idx, 1):
            hfo_y_arrays.append((both_band_seeg_window[:, hfo_idx], f"Both Bands Ch. {min_channel_idx + hfo_idx}"))


# Create the SEEG Voltage plot
hfo_plot = create_fig(
    title="SEEG Voltage dynamics of Filtered Both Bands", 
    x_axis_label='time (ms)', 
    y_axis_label='Voltage (μV)',
    x=x, 
    y_arrays=hfo_y_arrays, 
    sizing_mode="stretch_both", 
    tools="pan, box_zoom, wheel_zoom, hover, undo, redo, zoom_in, zoom_out, reset, save",
    tooltips="Data point @x: @y",
    legend_location="top_right",
    legend_bg_fill_color="navy",
    legend_bg_fill_alpha=0.1,
    # y_range=Range1d(-0.05, 1.05)
)

# If there are more than 30 channels, hide the legend
if len(hfo_y_arrays) > 30:
    # Hide the legend
    hfo_plot.legend.visible = False

## Add Box Annotations to the plot to identify the marked HFOs (ground truth)

In [87]:
from bokeh.models import BoxAnnotation
# from utils.line_plot import color_map

show_markers = False    # Boolean to show the markers

color_map = {                  
    'Spike': 'red',
    'Fast-Ripple': 'blue',
    'Ripple': 'green',  
    'Spike+Ripple': 'yellow',
    'Spike+Fast-Ripple': 'pink',
    'Ripple+Fast-Ripple': 'cyan',
    'Spike+Ripple+Fast-Ripple': 'black'
}

confidence_range = 100          # TODO: Check this value. When the duration is missing (0), we consider the 200ms window around the marked position 
visited_markers = {}    # Avoid inserting multiple boxes for the same marker (only one of each label)
use_visited = False     # Boolean controlling if we remove duplicate markers
plot_instant = True     # Boolean to plot the markers as instant events or as boxes
instant_width = 100 # 20       # Width of the instant event for visualization purposes

if show_markers:
    for ch_idx in channels_used:
        channel_markers = markers[ch_idx]
        # print("channel_markers", channel_markers)
        for idx2, marker in enumerate(channel_markers):
            # print("marker:", marker)
            
            if use_visited:
                # Check if the marker has already been visited and skip it if it has
                if marker['position'] in visited_markers:
                    visited_labels = visited_markers[marker['position']]    # Get the labels that already have an annotation for this position
                    if marker['label'] in visited_labels:
                        # print("Skipping marker", marker['position'], marker['label'])
                        continue    # Skip this marker
                    else:
                        visited_labels.append(marker['label'])  # Add the label to the visited labels
                else:
                    visited_markers[marker['position']] = [marker['label']] # Add the marker to the visited markers

            # Add a box annotation for each marker
            has_duration = marker['duration'] > 0
            
            confidence_constant = 0 if plot_instant or has_duration else confidence_range

            left = marker['position'] - confidence_constant
            right = marker['position'] + confidence_constant + instant_width
            box_color = color_map[marker['label']]  # Choose a color according to the label
            
            # if left < min_t or right > max_t:
            #     continue    # Skip this marker
            

            box = BoxAnnotation(left=left, right=right, fill_color=box_color, fill_alpha=0.35)
            # print("Added marker for channel: ", ch_idx, " at position: ", left)
            hfo_plot.add_layout(box)

## Show the Plot

In [88]:
import bokeh.plotting as bplt

showPlot = True
if showPlot:
    bplt.show(hfo_plot)

## Export the plot to a file

In [89]:
export = True
CH_SUFFIX = f"{min_channel_idx}" if is_single_channel else f"{min_channel_idx}-{max_channel_idx - 1}"
file_name = f"{OUT_FILE_PREFIX}{CH_SUFFIX}"

if export:
    file_path = f"plots{OUTPUT_PATH}/{file_name}.html"

    # Customize the output file settings
    bplt.output_file(filename=file_path, title="SEEG Data - Filtered Voltage dynamics across time")

    # Save the plot
    bplt.save(hfo_plot)

## Export the filtered signals to a numpy file

### Get the relevant Markers

In [90]:
# Save the relevant markers in a variable
relevant_markers = markers[min_channel_idx:max_channel_idx]
preview_np_array(relevant_markers, "Relevant Markers", edge_items=3)

Relevant Markers Shape: (30,).
Preview: [array([('Ripple',   522.9492, 0.), ('Fast Ripple',   523.9258, 0.),
        ('Ripple',  2225.586 , 0.), ..., ('Ripple', 61297.363 , 0.),
        ('Fast Ripple', 62202.637 , 0.), ('Ripple', 62207.52  , 0.)],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])
 array([('Ripple',   522.9492, 0.), ('Fast Ripple',   523.9258, 0.),
        ('Ripple',  2225.586 , 0.), ..., ('Ripple', 61297.363 , 0.),
        ('Fast Ripple', 62202.637 , 0.), ('Ripple', 62207.52  , 0.)],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])
 array([('Ripple',   522.9492, 0.), ('Ripple',  2225.586 , 0.),
        ('Fast Ripple',  2230.957 , 0.), ..., ('Ripple', 61297.363 , 0.),
        ('Fast Ripple', 62202.637 , 0.), ('Ripple', 62207.52  , 0.)],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])
 ...
 list([array([],
       dtype=[('label', '<U64'), ('position', '<f4'), ('duration', '<f4')])])
 list([a

In [91]:
from utils.input import RIPPLE_BAND_FILENAME, FR_BAND_FILENAME, BOTH_BAND_FILENAME

EXPORT_FILTERED_SIGNAL = True
if EXPORT_FILTERED_SIGNAL:
    # Export the filtered signals
    np.save(f"{PATH_TO_FILE}results{OUTPUT_PATH}/{file_name}_{RIPPLE_BAND_FILENAME}_band.npy", ripple_band_seeg_window)
    np.save(f"{PATH_TO_FILE}results{OUTPUT_PATH}/{file_name}_{FR_BAND_FILENAME}_band.npy", fr_band_seeg_window)
    np.save(f"{PATH_TO_FILE}results{OUTPUT_PATH}/{file_name}_{BOTH_BAND_FILENAME}_band.npy", both_band_seeg_window)

    # Export the markers
    np.save(f"{PATH_TO_FILE}results{OUTPUT_PATH}/{file_name}_markers.npy", relevant_markers)